In [34]:
import sys
print(sys.version)


3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]


In [35]:
import numpy as np
np.random.seed(1)

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from math import sqrt
import datetime as dt
import time
plt.style.use('ggplot')

In [36]:
from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y')

file = 'main_ds_nlp_weather_opt.csv'
df = pd.read_csv(file,parse_dates = True,index_col='Date', infer_datetime_format=True)
#print(df)
#print(df.columns)

D:\Work\anaconda3\envs\tensorflow-gpu\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '13/01/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
D:\Work\anaconda3\envs\tensorflow-gpu\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '14/01/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
D:\Work\anaconda3\envs\tensorflow-gpu\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '15/01/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
D:\Work\anaconda3\envs\tensorflow-gpu\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '16/01/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  re

In [37]:
#df.corr()['ENS']


def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=44):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

#print("Top 10 correlations")
#print(get_top_abs_correlations(df, 44))

In [24]:
print(df.describe().ENS)
#df.drop(df[df['Volume']==0].index, inplace = True)

count    1258.000000
mean        0.010421
std         1.296699
min        -9.620003
25%        -0.599998
50%         0.030003
75%         0.680000
max         5.430000
Name: ENS, dtype: float64


In [25]:
# Setting up an early stop
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=80,  verbose=1, mode='min')
callbacks_list = [earlystop]

In [26]:
#MODEL BUILD
def fit_model(train,val,timesteps,hl,lr,batch,epochs):
    X_train = []
    Y_train = []
    X_val = []
    Y_val = []
    print("Fit Model Initialized")
    # Loop for training data
    for i in range(timesteps,train.shape[0]):
        X_train.append(train[i-timesteps:i])
        Y_train.append(train[i][0])
    X_train,Y_train = np.array(X_train),np.array(Y_train)
  
    # Loop for val data
    for i in range(timesteps,val.shape[0]):
        X_val.append(val[i-timesteps:i])
        Y_val.append(val[i][0])
    X_val,Y_val = np.array(X_val),np.array(Y_val)
    print("Adding layers")
    # Adding Layers to the model
    model = Sequential()
    model.add(LSTM(X_train.shape[2],input_shape = (X_train.shape[1],X_train.shape[2]),return_sequences = True,
                   activation = 'relu'))
    for i in range(len(hl)-1):        
        model.add(LSTM(hl[i],activation = 'relu',return_sequences = True))
    model.add(LSTM(hl[-1],activation = 'relu'))
    model.add(Dense(1))
    model.compile(optimizer = optimizers.Adam(learning_rate = lr), loss = 'mean_squared_error') #SGD #RMSprop
    #print(model.summary())
    print("Training Data")
    # Training the data
    history = model.fit(X_train,Y_train,epochs = epochs,batch_size = batch,validation_data = (X_val, Y_val),verbose = 1,
                        shuffle = False, callbacks=callbacks_list)
    model.reset_states()
    print("Returning Values")
    return model, history.history['loss'], history.history['val_loss']


In [27]:
# EVALUATION
def evaluate_model(model,test,timesteps):
    X_test = []
    Y_test = []

    for i in range(timesteps,test.shape[0]):
        X_test.append(test[i-timesteps:i])
        Y_test.append(test[i][0])
    X_test,Y_test = np.array(X_test),np.array(Y_test)
    #print(X_test.shape,Y_test.shape)

    Y_hat = model.predict(X_test)
    mse = mean_squared_error(Y_test,Y_hat)
    rmse = sqrt(mse)
    mape = mean_absolute_percentage_error(Y_test,Y_hat)
    r = r2_score(Y_test,Y_hat)
    return mape, rmse, r, Y_test, Y_hat

In [28]:
# Plotting the predictions
def plot_data(Y_test,Y_hat):
    plt.plot(Y_test,c = 'r')
    plt.plot(Y_hat,c = 'y')
    plt.xlabel('Day')
    plt.ylabel('Price')
    plt.title('Stock Prediction using LSTM')
    plt.legend(['Actual Value','Predicted Value'],loc = 'lower right')
    plt.show()

In [29]:
# Plotting the training errors
def plot_error(train_loss,val_loss):
    plt.plot(train_loss,c = 'r')
    plt.plot(val_loss,c = 'b')
   # plt.yticks(np.arange(max(min(train_loss),min(val_loss))-0.05, max(max(train_loss),max(val_loss))+0.05, 0.05))
    #plt.xticks(0, 14, 1)
    plt.ylabel('Loss')
    plt.legend(['Training_Set','Validation_Set'],loc = 'upper right')
    plt.show()

In [30]:
# Extracting the series
#COLS=['ENS', 'CBAT', 'TIA', 'ALB', 'LAC', 'SQM', 'ALB_sentiment_nltk',
#       'CBAK_sentiment_nltk', 'ENS_sentiment_nltk', 'LAC_sentiment_nltk',
#       'SQM_sentiment_nltk', 'TIA_sentiment_nltk', 'lithium_sentiment_nltk',
#       'ALB_sentiment_TextBlob', 'CBAK_sentiment_TextBlob',
#       'ENS_sentiment_TextBlob', 'LAC_sentiment_TextBlob',
#       'SQM_sentiment_TextBlob', 'TIA_sentiment_TextBlob',
#       'lithium_sentiment_TextBlob', 'ALB_1_temp', 'ALB_1_humidity',
#       'ALB_1_windspeed', 'ALB_1_conditions', 'ALB_2_temp', 'ALB_2_humidity',
#       'ALB_2_windspeed', 'ALB_2_conditions', 'LAC_1_temp', 'LAC_1_humidity',
#       'LAC_1_windspeed', 'LAC_1_conditions', 'LAC_2_temp', 'LAC_2_humidity',
#       'LAC_2_windspeed', 'LAC_2_conditions', 'SQM_1_temp', 'SQM_1_humidity',
#      'SQM_1_windspeed', 'SQM_1_conditions', 'SQM_2_temp', 'SQM_2_humidity',
#      'SQM_2_windspeed', 'SQM_2_conditions']

series = df[['ENS', 'ALB', 'SQM','ALB_sentiment_nltk', 'ENS_sentiment_nltk',
       'SQM_sentiment_nltk', 'lithium_sentiment_nltk','ALB_1_windspeed', 'ALB_1_conditions', 'ALB_2_temp', 'ALB_2_humidity',
       'ALB_2_windspeed', 'ALB_2_conditions', 'SQM_1_temp', 'SQM_1_humidity',
      'SQM_1_windspeed', 'SQM_1_conditions', 'SQM_2_temp', 'SQM_2_humidity',
      'SQM_2_windspeed', 'SQM_2_conditions']]
      # Picking the series with high correlation
print(series.shape)
print(series.tail())

(1258, 21)
                 ENS       ALB       SQM  ALB_sentiment_nltk  \
Date                                                           
2019-12-24 -0.280006 -0.110001 -0.040001                 0.0   
2019-12-26  0.370003  0.169998  0.420000                 0.0   
2019-12-27 -0.239998 -0.479996 -1.289999                 0.0   
2019-12-30  0.070000 -0.080002 -0.510000                 0.0   
2019-12-31 -0.750000  1.110001  0.350000                 0.0   

            ENS_sentiment_nltk  SQM_sentiment_nltk  lithium_sentiment_nltk  \
Date                                                                         
2019-12-24                 0.0                 0.0                     0.0   
2019-12-26                 0.0                 0.0                     0.0   
2019-12-27                 0.0                 0.0                     0.0   
2019-12-30                 0.0                 0.0                     0.0   
2019-12-31                 0.0                 0.0                     0

In [31]:
# Train Val Test Split
#train_start = dt.date(2015,1,2)
#train_end = dt.date(2019,5,31)

#train_data = series.loc[train_start:train_end]
#train_data = series.loc['2015-1-2':'2019-1-1']
#train_data = series.loc['2015-1-2':'2019-5-31']
train_data = series.loc['2015-1-2':'2019-9-30']
#val_start = dt.date(2019,6,1)
#val_end = dt.date(2019,11,31)
#val_data = series.loc[val_start:val_end]
#val_data = series.loc['2019-1-1':'2019-5-31']
#val_data = series.loc['2019-5-31':'2019-8-31']
val_data = series.loc['2019-9-30':'2019-11-15']
#test_start = dt.date(2019,12,1)
#test_end = dt.date(2019,12,31)
#test_data = series.loc[test_start:test_end]
#test_data = series.loc['2019-5-31':'2019-12-31']
#test_data = series.loc['2019-8-31':'2019-12-31']
test_data = series.loc['2019-11-15':'2019-12-31']

#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

print(train_data.shape,val_data.shape,test_data.shape)

(1193, 21) (36, 21) (31, 21)


C:\Users\gaurs\AppData\Local\Temp\ipykernel_30504\955238725.py:8: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  train_data = series.loc['2015-1-2':'2019-9-30']


In [32]:
#Normalisation
sc = MinMaxScaler()
train = sc.fit_transform(train_data)
val = sc.transform(val_data)
test = sc.transform(test_data)
print(train.shape,val.shape,test.shape)

timesteps = 60
hl = [40,35]
lr = 1e-3
batch_size =64
num_epochs = 4

In [33]:
print("Starting Model Execution")
model,train_error,val_error = fit_model(train,val,timesteps,hl,lr,batch_size,num_epochs)


Starting Model Execution
Fit Model Initialized


KeyError: 60

In [ ]:
%matplotlib inline
print("Plotting Error")
plot_error(train_error,val_error)

In [ ]:
mape, rmse, r2_value,true,predicted = evaluate_model(model,test,timesteps)
print('MAPE = {}'.format(mape))
print('RMSE = {}'.format(rmse))
#print('R-Squared Score = {}'.format(r2_value))
plot_data(true,predicted)